In [1]:
import tensorflow as tf
import numpy as np
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32).reshape(-1, 28*28) / 255.0
x_test = x_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
x_valid, x_train = x_train[:5000], x_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [2]:
def shuffle_batch(x, y, batch_size):
    rnd_idx = np.random.permutation(len(x))
    n_batches = len(x) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        x_batch, y_batch = x[batch_idx], y[batch_idx]
        yield x_batch, y_batch

In [3]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(0)
np.random.seed(0)

In [4]:
lr = 1e-2
epoch = 20
batch_size = 200
l2_regularization_constant = 1e-3 

In [5]:
n_inputs = 28 * 28  
n_hidden1 = 300
n_outputs = 10

x = tf.placeholder(tf.float32, shape=(None, n_inputs), name="x")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(x, n_hidden1, activation=tf.nn.relu, name="hidden1")
    logits = tf.layers.dense(hidden1, n_outputs, name="outputs")

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [6]:
g = tf.get_default_graph()

In [7]:
op_list = g.get_operations()
for op in op_list:
    print(op)

name: "x"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 784
      }
    }
  }
}

name: "y"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "shape"
  value {
    shape {
      unknown_rank: true
    }
  }
}

name: "hidden1/kernel/Initializer/random_uniform/shape"
op: "Const"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@hidden1/kernel"
    }
  }
}
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
        dim {
          size: 2
        }
      }
      tensor_content: "\020\003\000\000,\001\000\000"
    }
  }
}

name: "hidden1/kernel/Initializer/random_uniform/min"
op: "Const"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@hidden1/kernel"
    }
  }
}
attr {
  key: "dtype"
  value {
    type: DT_F

In [8]:
op_name_list = []
for op in op_list:
    op_name_list.append(op.name)
for op_name in op_name_list:
    print(op_name)

x
y
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
dnn/hidden1/Relu
outputs/kernel/Initializer/random_uniform/shape
outputs/kernel/Initializer/random_uniform/min
outputs/kernel/Initializer/random_uniform/max
outputs/kernel/Initializer/random_uniform/RandomUniform
outputs/kernel/Initializer/random_uniform/sub
outputs/kernel/Initializer/random_uniform/mul
outputs/kernel/Initializer/random_uniform
outputs/kernel
outputs/kernel/Assign
outputs/kernel/read
outputs/bias/Initializer/zeros
outputs/bias
outputs/bias/Assign
outputs/bias/read
dn

In [9]:
print(type(tf.GraphKeys.TRAINABLE_VARIABLES))
print(tf.GraphKeys.TRAINABLE_VARIABLES)

<class 'str'>
trainable_variables


In [10]:
print(type(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)))
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

<class 'list'>
[<tf.Variable 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>, <tf.Variable 'hidden1/bias:0' shape=(300,) dtype=float32_ref>, <tf.Variable 'outputs/kernel:0' shape=(300, 10) dtype=float32_ref>, <tf.Variable 'outputs/bias:0' shape=(10,) dtype=float32_ref>]


In [11]:
# w1 = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
# w2 = tf.get_default_graph().get_tensor_by_name("outputs/kernel:0")

In [12]:
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    entropy_loss = tf.reduce_mean(entropy, name="entropy_loss")
    l2_loss_list = [tf.nn.l2_loss(var)
                    for var in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
                    if "kernel" in var.name]
    l2_loss = tf.add_n(l2_loss_list)
    loss = tf.identity(entropy_loss + l2_regularization_constant * l2_loss, name="loss")

In [13]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    opt = tf.train.GradientDescentOptimizer(lr)
    train = opt.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [14]:
with tf.Session() as sess:
    init.run()
    for i in range(epoch):
        for x_batch, y_batch in shuffle_batch(x_train, y_train, batch_size):
            sess.run(train, feed_dict={x: x_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={x: x_valid, y: y_valid})
        print(i, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./save/my_model_final.ckpt")

0 Validation accuracy: 0.8268
1 Validation accuracy: 0.8668
2 Validation accuracy: 0.8804
3 Validation accuracy: 0.8936
4 Validation accuracy: 0.9022
5 Validation accuracy: 0.906
6 Validation accuracy: 0.9104
7 Validation accuracy: 0.9144
8 Validation accuracy: 0.9156
9 Validation accuracy: 0.917
10 Validation accuracy: 0.9186
11 Validation accuracy: 0.92
12 Validation accuracy: 0.9228
13 Validation accuracy: 0.9218
14 Validation accuracy: 0.9248
15 Validation accuracy: 0.9252
16 Validation accuracy: 0.9274
17 Validation accuracy: 0.9286
18 Validation accuracy: 0.9298
19 Validation accuracy: 0.9324
